In [108]:
import angr

In [109]:
proj = angr.Project("foo.o", load_debug_info=True)
print(proj.arch)
t = angr.types.parse_type('int')
t.arch = proj.arch
# print(inspect.getmembers(t))
print(dir(t))
print(t.base)
print(t.c_repr)
print(t.label)
print(t.signed)
t2 = t.with_arch(proj.arch)
print(t2.alignment)

WARNING  | 2023-02-24 14:22:16,654 | cle.loader     | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


<Arch AMD64 (LE)>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_arch', '_base_name', '_can_refine_int', '_fields', '_init_str', '_refine', '_refine_dir', '_size', '_with_arch', 'alignment', 'arch', 'base', 'c_repr', 'copy', 'extract', 'extract_claripy', 'label', 'signed', 'size', 'store', 'with_arch']
True
<bound method SimTypeInt.c_repr of int>
None
True
4


In [4]:
angr.types.parse_type('struct aa {int x; long y;}').fields

OrderedDict([('x', int), ('y', long)])

In [9]:
dir(proj.factory.cc()) # calling conventions. Interesting


['ARCH',
 'ARG_REGS',
 'ArgSession',
 'CALLEE_CLEANUP',
 'CALLER_SAVED_REGS',
 'FP_ARG_REGS',
 'FP_RETURN_VAL',
 'OVERFLOW_FP_RETURN_VAL',
 'OVERFLOW_RETURN_VAL',
 'RETURN_ADDR',
 'RETURN_VAL',
 'STACKARG_SP_BUFF',
 'STACKARG_SP_DIFF',
 'STACK_ALIGNMENT',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_classify',
 '_combine_classes',
 '_flatten',
 '_match',
 '_standardize_value',
 'arch',
 'arg_locs',
 'arg_session',
 'find_cc',
 'fp_args',
 'get_arg_info',
 'get_args',
 'guess_prototype',
 'int_args',
 'is_fp_arg',
 'is_fp_value',
 'memory_args',
 'next_arg',
 'return_addr',
 'return_in_implicit_outparam',
 'return_val',
 'set_return_val',
 'setu

Hmm. Here's an idea. Literally use angr to write specs?
mem.long.yada == foo

https://api.angr.io/angr.html#module-angr.calling_conventions


state 

unroll_loop parameter. That's interesting.

In [118]:
st = proj.factory.entry_state()
st = proj.factory.entry_state(add_options={angr.options.LAZY_SOLVES, angr.options.SYMBOLIC, angr.options.SYMBOLIC_INITIAL_VALUES})
st.regs.rax
st.mem[0x601048].long
st.registers
st.memory
list(st.solver.get_variables(""))
st.regs.rax.to_claripy()
st.registers.

In [19]:
import claripy
s = claripy.Solver()

In [21]:
s.add(st.regs.rax == st.regs.rdi)
()

(<Bool False>,)

In [88]:
#https://api.angr.io/angr.html#angr.state_plugins.debug_variables.SimDebugVariablePlugin
#p = angr.Project("various_variables", load_debug_info=True)
proj.kb.dvars.load_from_dwarf()
state = proj.factory.call_state(0x400000) # navigate to the state you want
#state.dvars.get_variable("x").deref.mem
state.dvars.dwarf_cfa

WARNING  | 2023-02-23 14:36:39,907 | angr.calling_conventions | Guessing call prototype. Please specify prototype.
WARNING  | 2023-02-23 14:36:39,919 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2023-02-23 14:36:39,921 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2023-02-23 14:36:39,922 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2023-02-23 14:36:39,923 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2023-02-23 14:36:39,925 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these

<BV64 reg_rbp_63_64{UNINITIALIZED} + 0x10>

In [42]:
angr.analyses.decompiler

<module 'angr.analyses.decompiler' from '/home/philip/.local/lib/python3.10/site-packages/angr/analyses/decompiler/__init__.py'>

In [104]:
#proj.analyses.Decompiler()
sym = proj.loader.find_symbol("foo")
#proj.load_function(sym)
cfg = proj.analyses.CFGFast()
cfg.normalize()
foo_func = cfg.kb.functions["foo"]
foo_func.normalize()
decomp = proj.analyses.Decompiler(foo_func)
decomp.codegen.cfunc.c_repr()


WARNING  | 2023-02-23 14:52:21,986 | angr.analyses.cfg.cfg_base | "auto_load_libs" is enabled. With libraries loaded in project, CFG will cover libraries, which may take significantly more time than expected. You may reload the binary with "auto_load_libs" disabled, or specify "regions" to limit the scope of CFG recovery.


'long long foo(unsigned long a0, unsigned long a1)\n{\n    char v0;  // [bp-0x20]\n\n    v0 = a1;\n    return a0 + 3;\n}\n'

In [94]:
t = angr.sim_type.parse_signature("int foo(int x, char y)")
t.arg_names
t.args
t.

[int, char]

In [107]:

proj.kb.defs
proj.kb.propagations
# angr.knowledge_plugins.patches.PatchManager
proj.kb.functions["foo"] # need to fun CFG first

<Function foo (0x400000)>

In [ ]:
angr.analyses.bindiff.FunctionDiff
https://api.angr.io/angr.html#angr.analyses.bindiff.FunctionDiff
